In [1]:
# set up the notebook
%load_ext autoreload
%autoreload 2
import sys
import os
import logging
logging.basicConfig(level=logging.ERROR)
from functools import partial
import re
import openai
import json
from pdfminer.high_level import extract_text

with open("./credential.json", "r") as f:
    credential = json.load(f)["openai"]
openai.api_key = credential

In [81]:
# FILL IN MANUALLY
# paper title
title = "Harmonizing the object recognition strategies of deep neural networks with humans"
# paper should be stored as papers/{paper_index}.pdf
paper_index = 11

# maps checklist questions to the corresponding sections that should be used as context
question_section_mapping = {
    "1b": ["conclusion"],
    "1c": ["conclusion"],
    "2a": ["methods"],
    "2b": ["methods"],
    "3a": ["experiments"],
    "3b": ["experiments"],
    "3c": ["experiments", "appendix"],
    "3d": ["experiments"],
    "4a": ["methods"],
    "4b": ["methods"],
    "4c": ["introduction", "methods"],
    "4d": ["methods"],
    "4e": ["methods"],
    "5a": ["methods", "appendix"],
    "5b": ["methods", "appendix"],
    "5c": ["methods"]
}

In [91]:
# get the paper and appendix text
file = f'papers/{paper_index}.pdf'
text = extract_text(file)
supplemental_file = f'papers/{paper_index}supp.pdf'
supplemental_text = extract_text(supplemental_file)

# get the sections; make sure to remove the checklist section using re.search
abstract = re.search(r"Abstract(.*?)Introduction", text, re.DOTALL).group(1)
introduction = re.search(r"Introduction(.*?)Do DNNs explain human visual per-", text, re.DOTALL).group(1)
methods = re.search(r"3 Methods(.*?)4 Results", text, re.DOTALL).group(1)
experiments = re.search(r"4 Results(.*?)5 Conclusion", text, re.DOTALL).group(1) + " higher diameter and value norm."
conclusion = re.search(r"5 Conclusion(.*?)Acknowledgments and Disclosure of Funding", text, re.DOTALL).group(1)
appendix = re.search(r"1 Psychophyics(.*?)References", supplemental_text, re.DOTALL).group(1)
# remove all <latexit text
methods = re.sub(r"<latexit.*?>.*?</latexit>", "", methods)
experiments = re.sub(r"<latexit.*?>.*?</latexit>", "", experiments)
# remove all (cid:xx)
introduction = re.sub(r"\(cid:.*?\)", "", introduction)
appendix = re.sub(r"\(cid:.*?\)", "", appendix)

print("abstract", len(abstract))
print("introduction", len(introduction))
print("methods", len(methods))
print("experiments", len(experiments))
print("conclusion", len(conclusion))
print("appendix", len(appendix))

abstract 1418
introduction 4452
methods 4690
experiments 14653
conclusion 3857
appendix 11803


In [69]:
# Only include when appendix is too large to fit into the token limit, thus needing to break it into sections.
appendix_1 = re.search(r"S1.2 Main Assumptions(.*?)S1.5 Supporting Lemmata", appendix, re.DOTALL).group(1)
appendix_2 = re.search(r"S2 Application to(.*?)Additional Experiments", appendix, re.DOTALL).group(1) + " unsurprising given the MDP structure."
appendix_3 = re.search(r"Additional Experiments(.*?)Figure S8", appendix, re.DOTALL).group(1)

print("appendix_A", len(appendix_1))
print("appendix_D", len(appendix_2))
print("appendix_E", len(appendix_3))

appendix_A 6488
appendix_D 2710
appendix_E 2559


In [92]:
# define the save dict
save_dict = {}
save_dict["paper_index"] = paper_index
save_dict["title"] = title
save_dict["abstract"] = abstract
save_dict["introduction"] = introduction
save_dict["methods"] = methods
save_dict["experiments"] = experiments
save_dict["conclusion"] = conclusion
save_dict["appendix"] = appendix

In [71]:
# Only include when appendix is too large to fit into the token limit (as above).
save_dict["appendix_1"] = appendix_1
save_dict["appendix_2"] = appendix_2
save_dict["appendix_3"] = appendix_3

In [93]:
# construct the system prompts
system_prompt = f"You are a computer science researcher currently reviewing a paper titled \"{title}\" for the NeurIPS computer science conference. Your goal is to try to be as objective and truthful as possible in your answers about the paper provided. Your reviews will be used for causal reasoning in determining the quality of the paper."

def turn_into_input_format(role, prompt):
    # role: "system", "user", or "assistant"
    return {"role": role, \
            "content": prompt}

system_prompt = turn_into_input_format("system", system_prompt)

In [94]:
# define checklist questions
checklist_questions = {
    "1b": "Do the authors describe the limitations of their work?",
    "1c": "Do the authors discuss any potential negative societal impacts of their work?",
    "2a": "If the authors include theoretical results, do the authors state the full set of assumptions of all theoretical results?",
    "2b": "If the authors include theoretical results, do the authors include complete proofs of all theoretical results?",
    "3a": "If the authors ran experiments, do the authors include the code, data, and instructions needed to reproduce the main experimental results (either in the supplemental material or as a URL)?",
    "3b": "If the authors ran experiments, do the authors specify all the training details (e.g., data splits, hyperparameters, how they were chosen)?",
    "3c": "If the authors ran experiments, do the authors report error bars (e.g., with respect to the random seed after running experiments multiple times)?",
    "3d": "If the authors ran experiments, do the authors include the total amount of compute and the type of resources used (e.g., type of GPUs, internal cluster, or cloud provider)?",
    "4a": "If the authors use existing assets (e.g., code, data, models), do the authors cite the creators?",
    "4b": "If the authors use existing assets (e.g., code, data, models) or curate/release new assets, do the authors mention the license of the assets?",
    "4c": "If the authors curate/release new assets (e.g., code, data, models), do the authors include any new assets either in the supplemental material or as a URL?",
    "4d": "If the authors curate/release new assets (e.g., code, data, models), do the authors discuss whether and how consent was obtained from people whose data they are using/curating?",
    "4e": "If the authors curate/release new assets (e.g., code, data, models), do the authors discuss whether the data they are using/curating contains personally identifiable information or offensive content?",
    "5a": "If the authors used crowdsourcing or conducted research with human subjects, do the authors include the full text of instructions given to participants and screenshots, if applicable?",
    "5b": "If the authors used crowdsourcing or conducted research with human subjects, do the authors describe any potential participant risks, with links to Institutional Review Board (IRB) approvals, if applicable?",
    "5c": "If the authors used crowdsourcing or conducted research with human subjects, do the authors include the estimated hourly wage paid to participants and the total amount spent on participant compensation?"
}

In [95]:
# build the full prompts based on the individual parts
prompts = {}
for question in question_section_mapping.keys(): # e.g. q1c -> ["abstract"]
    sections = question_section_mapping[question]

    prompt_prefix = ""
    for section in sections:
        prompt_prefix += f'The following is the {section} section of the paper you are reviewing:\n'
        prompt_prefix += save_dict[section] + "\n\n"
    prompt_question = "Based on the section(s), please answer the following question with yes, no, or n/a and provide a brief justification for your answer.\nQuestion: "
    prompt = prompt_prefix + prompt_question + checklist_questions[question]
    prompts[question] = turn_into_input_format("user", prompt) # put the inputs into input format for openAI API

In [96]:
# query the GPT-4 API
def query_problem(system_prompt, alternative, model="gpt-4", max_tokens=150, temperature=1, top_p=1, n=1, verbose=False):
    # alternative is the user prompt
    messages = [system_prompt, alternative] # first message in the chain of messages
    if verbose:
        print("messages:", messages)
    
    completion1 = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p, 
        n=n
    )

    return [completion1["choices"]]

In [97]:
# get the responses across multiple problems
import time
def get_responses(verbose=False, start_point=None):
    # start_point is 0-indexed, with the order (pilot_questions, sys_index, alt_index)
    params = (1, 1) # (temperature, top_p)
    questions = list(prompts.keys())
    
    for problem_index in range(len(questions)):
        problem = questions[problem_index]
        prompt = prompts[problem]

        if start_point is not None:
            if (start_point > problem_index): continue

        print(f'problem {problem_index+1}/{len(questions)}')
            
        answers = query_problem(system_prompt, prompt, temperature=params[0], top_p=params[1], n=3, verbose=verbose)
        # save the answers to a file, create the directory if necessary
        if not os.path.exists(f'data/{paper_index}'):
            os.makedirs(f'data/{paper_index}')
        
        with open(f'data/{paper_index}/{problem}.json', "w") as f:
            json.dump(answers, f)
        time.sleep(10) # OpenAI limit is like 40000 tokens/min

In [77]:
get_responses(verbose=False, start_point=0) # the index of the next one that hasn't been queried

problem 2/8
problem 3/8
problem 4/8
problem 5/8
problem 6/8
problem 7/8
problem 8/8


In [98]:
# add the system prompts to the save dict
save_dict["system_prompts"] = [system_prompt]

# add the prompts for each question to the save dict
save_dict["prompts"] = prompts

# save the save_dict
with open(f'data/{paper_index}/save_dict.json', "w") as f:
    json.dump(save_dict, f)